In [26]:
import cv2
import pytesseract
from gtts import gTTS
import os
import time

In [27]:
import pytesseract
print(pytesseract.__version__)

0.3.13


In [28]:
# Configure Tesseract OCR (Update this path for Windows users)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # Change if needed

In [29]:
def text_to_speech(text, lang='en'):
    """
    Convert detected text to speech using gTTS.
    """
    tts = gTTS(text=text, lang=lang, slow=False)
    tts.save("output.mp3")  # Save audio file
    os.system("start output.mp3")  # Play audio (Windows) | 'afplay' for Mac, 'mpg321' for Linux

In [30]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

last_spoken = ""  # Store last spoken text to avoid repetition

In [33]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Convert to grayscale (better for OCR)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding for better OCR on handwriting
    gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                 cv2.THRESH_BINARY, 31, 2)

    # Use OCR to detect handwritten & printed text
    text_data = pytesseract.image_to_data(gray, output_type=pytesseract.Output.DICT)

    for i in range(len(text_data['text'])):
        if int(text_data['conf'][i]) > 50:  # Confidence threshold (lower for handwriting)
            x, y, w, h = (text_data['left'][i], text_data['top'][i], 
                          text_data['width'][i], text_data['height'][i])
            detected_text = text_data['text'][i].strip()

            if detected_text:
                # Draw red bounding box around detected text
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

                # Draw label below the text
                cv2.rectangle(frame, (x, y + h), (x + w, y + h + 25), (0, 0, 255), -1)
                cv2.putText(frame, detected_text, (x + 5, y + h + 20), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

                # Speak detected text only if different from last spoken text
                if detected_text != last_spoken:
                    text_to_speech(detected_text)
                    last_spoken = detected_text  # Update last spoken text
                    time.sleep(2)  # Prevent repeating too fast

    # Show the frame
    cv2.imshow("Handwritten & Printed Text Reader with Speech", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

KeyboardInterrupt: 

In [ ]:
# Release webcam and close windows
cap.release()
cv2.destroyAllWindows()